In [30]:
from PIL import Image
import imagehash
import glob
import pandas as pd
from tqdm import tqdm
import torch
import torchvision
import os

from pytorch_fid import fid_score, inception

In [4]:
torch.cuda.device_count()

1

## Load 1k images

In [52]:
dir_1k = "/work/vajira/DL/checkpoints/results/test_1k"
dir_1k_ema = "/work/vajira/DL/checkpoints/results/test_1k_ema"
dir_1k_mr = "/work/vajira/DL/checkpoints/results/test_1k_mr"
dir_original = "/work/vajira/DATA/hyper_kvasir/data_new/unlabelled/data/unlabeled"
dir_original_128 = "/work/vajira/DATA/hyper_kvasir/data_new/unlabelled/extracted_3k_128x128"

# This each dir has 3k images
gan_gen_1k_dirs =[ "/work/vajira/DATA/hyper_kvasir/data_new/unlabelled/gan_generated_1k_from_0/new_test_005_unlabelled_GI_wihout_aug",
                    "/work/vajira/DATA/hyper_kvasir/data_new/unlabelled/gan_generated_1k_from_100/new_test_005_unlabelled_GI_wihout_aug",
                    "/work/vajira/DATA/hyper_kvasir/data_new/unlabelled/gan_generated_1k_from_200/new_test_005_unlabelled_GI_wihout_aug",
                    "/work/vajira/DATA/hyper_kvasir/data_new/unlabelled/gan_generated_1k_from_300/new_test_005_unlabelled_GI_wihout_aug",
                    "/work/vajira/DATA/hyper_kvasir/data_new/unlabelled/gan_generated_1k_from_400/new_test_005_unlabelled_GI_wihout_aug",
                    "/work/vajira/DATA/hyper_kvasir/data_new/unlabelled/gan_generated_1k_from_500/new_test_005_unlabelled_GI_wihout_aug",
                    "/work/vajira/DATA/hyper_kvasir/data_new/unlabelled/gan_generated_1k_from_600/new_test_005_unlabelled_GI_wihout_aug",
                    "/work/vajira/DATA/hyper_kvasir/data_new/unlabelled/gan_generated_1k_from_700/new_test_005_unlabelled_GI_wihout_aug",
                    "/work/vajira/DATA/hyper_kvasir/data_new/unlabelled/gan_generated_1k_from_800/new_test_005_unlabelled_GI_wihout_aug",
                    "/work/vajira/DATA/hyper_kvasir/data_new/unlabelled/gan_generated_1k_from_900/new_test_005_unlabelled_GI_wihout_aug",
                    "/work/vajira/DATA/hyper_kvasir/data_new/unlabelled/gan_generated_1k_from_1000/new_test_005_unlabelled_GI_wihout_aug"
                ]

In [50]:
fid_dims = [64, 192, 768, 2048]

## Extract random 3k images with 128 x 128

In [25]:
original_img_paths = glob.glob(dir_original + "/*")
dst_dir = "/work/vajira/DATA/hyper_kvasir/data_new/unlabelled/extracted_1k_128x128"

In [39]:
for i in tqdm(range(3000)):
    img = Image.open(original_img_paths[i])
    img = torchvision.transforms.functional.resize(img, 128)
    img = torchvision.transforms.functional.center_crop(img, 128)
    #img = img.resize((128, 128))
    img.save(os.path.join(dst_dir, str(i) + ".jpg" ), format="JPEG")

100%|██████████| 3000/3000 [00:48<00:00, 61.51it/s]


## Calculate FID for each checkpoint with different layers

In [58]:
fid_df = pd.DataFrame(columns=["chk_point", "FID_64", "FID_192", "FID_768", "FID_2048"])

In [59]:
for i, gan_dir in enumerate(gan_gen_1k_dirs): 
    chk_point = i * 100

    fid_64 = fid_score.calculate_fid_given_paths([gan_dir, dir_original_128], 64, torch.device("cuda:1"),dims=64)
    fid_192 = fid_score.calculate_fid_given_paths([gan_dir, dir_original_128], 64, torch.device("cuda:1"),dims=192)
    fid_768 = fid_score.calculate_fid_given_paths([gan_dir, dir_original_128], 64, torch.device("cuda:1"),dims=768)
    fid_2048 = fid_score.calculate_fid_given_paths([gan_dir, dir_original_128], 64, torch.device("cuda:1"),dims=2048)

    fid_df = fid_df.append({"chk_point": chk_point, "FID_64": fid_64, "FID_192": fid_192, "FID_768":fid_768, "FID_2048":fid_2048}, ignore_index=True)


100%|██████████| 47/47 [00:04<00:00, 10.31it/s]


In [60]:
fid_df

,chk_point,FID_64,FID_192,FID_768,FID_2048
0,0.0,39.109040,189.493765,2.615853,342.075087
1,100.0,1.770995,8.348001,0.303003,58.948974
2,200.0,1.661565,8.027057,0.297659,59.721490
3,300.0,1.657510,7.830963,0.267125,52.659655
4,400.0,1.280054,6.118289,0.242879,48.569359
5,500.0,1.226208,5.875927,0.237158,49.351194
6,600.0,1.597361,7.458629,0.262617,52.944055
7,700.0,1.382576,6.506347,0.236304,46.266849
8,800.0,1.193848,5.911193,0.231225,46.793082
9,900.0,0.653704,3.026018,0.201677,44.331041


In [ ]:
fid_df.to_csv("fid_scores.csv")

In [44]:
fid_score.calculate_fid_given_paths([gan_gen_1k_dirs[0], dir_original_128], 64, torch.device("cuda:1"),dims=192)

100%|██████████| 47/47 [00:02<00:00, 21.11it/s]


189.49376519130644

In [45]:
fid_score.calculate_fid_given_paths([gan_gen_1k_dirs[1], dir_original_128], 64, torch.device("cuda:1"),dims=192)

100%|██████████| 47/47 [00:02<00:00, 20.98it/s]


8.348000607243744

In [46]:
fid_score.calculate_fid_given_paths([gan_gen_1k_dirs[2], dir_original_128], 64, torch.device("cuda:1"),dims=192)

100%|██████████| 47/47 [00:02<00:00, 21.12it/s]


8.027057006413486

In [ ]:
fid_score.calculate_fid_given_paths([gan_gen_1k_dirs[0], dir_original_128], 64, torch.device("cuda:1"),dims=192)

In [48]:
fid_score.calculate_fid_given_paths([gan_gen_1k_dirs[8], dir_original_128], 64, torch.device("cuda:1"),dims=192)

100%|██████████| 47/47 [00:02<00:00, 21.04it/s]


5.911193134223296

In [47]:
fid_score.calculate_fid_given_paths([gan_gen_1k_dirs[9], dir_original_128], 64, torch.device("cuda:1"),dims=192)

100%|██████████| 47/47 [00:02<00:00, 21.13it/s]


3.02601831367744

In [10]:
img_1k_path_list = glob.glob(dir_1k + "/*")
img_1k_ema_path_list = glob.glob(dir_1k_ema + "/*")
img_1k_mr_path_list = glob.glob(dir_1k_mr + "/*a")

# Data frames to store hash values

In [23]:
df_1k = pd.DataFrame(columns=["img_path", "aHash", "pHash","dHash", "wHash", "clrHash" ])

In [25]:
aHash = imagehash.average_hash(Image.open(img_1k_path_list[0]))
pHash = imagehash.phash(Image.open(img_1k_path_list[0]))
dHash = imagehash.dhash(Image.open(img_1k_path_list[0]))
wHash = imagehash.whash(Image.open(img_1k_path_list[0]))
#cropHash = imagehash.crop_resistant_hash((Image.open(img_1k_path_list[0]))) # This is returning multiple hash values
clrHash = imagehash.colorhash((Image.open(img_1k_path_list[0])))

df_1k = df_1k.append({"img_path": img_1k_path_list[0],
                        "aHash": aHash, 
                        "pHash": pHash,
                        "dHash": dHash, 
                        "wHash": wHash, 
                        #"cropHash": cropHash,
                        "clrHash": clrHash
                    }, ignore_index=True)


In [26]:
df_1k

,img_path,aHash,pHash,dHash,wHash,cropHash,clrHash
0,/work/vajira/DL/checkpoints/results/test_1k/ge...,1e7f7f7e60c0c000,d1871c7117dc356c,f8f0f0c8d89888a2,1e7f7f7ee0e0c000,"f8f0f2e8d89888a2,f8f8f0f0f0f0f2ec,e8c8dc9898a0...",06e00000000


In [12]:
print(hash)

1e7f7f7e60c0c000


In [13]:
type(hash)

imagehash.ImageHash

In [ ]:
fid.